In [3]:
import pandas as pd
import os

folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    dfs.append(file_path)  # append the DataFrame to the list

In [4]:
len(dfs)

11

In [5]:
dfs

['/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/net_exchange.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/prices_brent_lng_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/.DS_Store',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/da_production_prognosis.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/weather_averaged_preprocessed_NO4.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/electricity_consumption.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/elspot_prices_dt_interpolated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/reservoir_levels.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/prices_osebx_updated.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/wind_settlement.csv',
 '/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/electricity_production.csv']

In [6]:
df1=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/elspot_prices_dt_interpolated.csv')
df1.set_index('datetime', inplace=True)
df1 = df1[~df1.index.duplicated()]

In [7]:
df1=df1['TromsÃ¸']
df1 = df1.to_frame('price')

In [8]:
df2=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/electricity_consumption.csv')
df2.set_index('datetime', inplace=True)
df2 = df2[~df2.index.duplicated()]

In [9]:
df2=df2['NO4']
df2.sort_index(ascending=True, inplace=True)
df2 = df2.to_frame('consumption')

In [10]:
df3=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/electricity_production.csv')
df3.set_index('datetime', inplace=True)
df3 = df3[~df3.index.duplicated()]

In [11]:
df3=df3['NO4']
df3.sort_index(ascending=True, inplace=True)
df3 = df3.to_frame('production')

In [12]:
df4=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/da_production_prognosis.csv')
df4.set_index('datetime', inplace=True)
df4 = df4[~df4.index.duplicated()]

In [13]:
df4=df4['NO4']
df4.sort_index(ascending=True, inplace=True)
df4 = df4.to_frame('da_production')

In [14]:
df5=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/net_exchange.csv')
df5.set_index('datetime', inplace=True)
df5 = df5[~df5.index.duplicated()]

In [16]:
df5.sort_index(ascending=True, inplace=True)
filtered_cols = df5.columns[df5.columns.str.contains('NO4')]
df5=df5[filtered_cols]
df5=df5.rename(columns={'NO3_NO4': 'exchange_NO3_NO4', 'NO4_SE1':'exchange_NO4_SE1', 'NO4_SE2':'exchange_NO4_SE2'})
df5

,exchange_NO3_NO4,exchange_NO4_SE1,exchange_NO4_SE2
datetime,,,
2019-12-30 00:00:00,28.0,505.0,113.0
2019-12-30 01:00:00,5.0,508.0,149.0
2019-12-30 02:00:00,-80.0,397.0,139.0
2019-12-30 03:00:00,-127.0,382.0,134.0
2019-12-30 04:00:00,-133.0,374.0,137.0
...,...,...,...
2023-04-07 19:00:00,NaN,NaN,NaN
2023-04-07 20:00:00,NaN,NaN,NaN
2023-04-07 21:00:00,NaN,NaN,NaN


In [17]:
df7=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/weather_averaged_preprocessed_NO4.csv')
# Convert the 'date' column to datetime
df7['referenceTime'] = pd.to_datetime(df7['referenceTime'])

# Extract the date part and assign it back to the 'date' column
df7['datetime'] = df7['referenceTime'].dt.date
df7['datetime'] = pd.to_datetime(df7['datetime'])
df7.set_index('datetime', inplace=True)
df7 = df7[~df7.index.duplicated()]
df7=df7.rename(columns={'mean(air_temperature P1D)': 'air_temperature', 'mean(wind_speed P1D)': 'wind_speed','sum(precipitation_amount P1D)':'precipitation_amount' })
df7.drop('referenceTime', axis=1, inplace=True)
df7 = df7.resample('H').ffill()


In [18]:
df8=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/reservoir_levels.csv')
# Convert the 'date' column to datetime
df8['datetime'] = pd.to_datetime(df8['Year/Week'] + '-1', format='%Y/%U-%w')
df8.set_index('datetime', inplace=True)
df8 = df8[~df8.index.duplicated()]
# 1. Convert string dates to datetime objects
start_date = pd.to_datetime('2019-12-29')
end_date = pd.to_datetime('2023-04-01')

# 2. Create new index using pd.date_range()
new_index = pd.date_range(start=start_date, end=end_date, freq='H')

# 3. Reindex DataFrame using new index
df8 = df8.reindex(pd.to_datetime(new_index))

df8 = df8.fillna(method='backfill')
df8 =df8.fillna(method='ffill')
df8=df8[['Capacity_GWh', 'Levels_GWh']]
df8=df8.rename(columns={'Capacity_GWh':'reservoir_capacity', 'Levels_GWh': 'reservoir_levels'})


In [19]:
df9=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/prices_brent_lng_updated.csv')
df9['datetime'] = pd.to_datetime(df9['Date'])
df9.set_index('datetime', inplace=True, drop=True)
df9 = df9[~df9.index.duplicated()]
df9 = df9.resample('H').ffill()
df9=df9[['BZ=F','NG=F']]
df9=df9.rename(columns={'BZ=F':'oil_price','NG=F':'gas_price' })

In [20]:
df10=pd.read_csv('/Users/markuswiikjensen/Desktop/Master-Data/NO4_data/prices_osebx_updated.csv')
df10['datetime'] = pd.to_datetime(df10['Date'])
df10.set_index('datetime', inplace=True, drop=True)
df10 = df10[~df10.index.duplicated()]
df10 = df10.resample('H').ffill()
df10 = df10.rename(columns={'Adj Close': 'osebx_price'})
df10 =df10['osebx_price']
df10 = df10.to_frame('osebx_price')

In [21]:
dfs = [df1, df2, df3, df4, df5, df7, df8, df9, df10]
df=df1.join(df2, on=df1.index, how='left')
df=df.join(df3,  on=df1.index, how='left')
df=df.join(df4,  on=df1.index, how='left')
df=df.join(df5,  on=df1.index, how='left')

In [22]:
df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S')

In [23]:
df=df.join(df7,  on='datetime', how='left')
df=df.join(df8,  on='datetime', how='left')
df=df.join(df9,  on='datetime', how='left')
df=df.join(df10,  on='datetime', how='left')

In [24]:
df

,price,consumption,production,da_production,exchange_NO3_NO4,exchange_NO4_SE1,exchange_NO4_SE2,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,28387.0,2192.0,2111.0,2100.0,298.0,308.0,42.0,-1.609396,6.846316,12.722727,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 01:00:00,28062.0,2357.0,3544.0,3452.0,646.0,-333.0,-163.0,-1.609396,6.846316,12.722727,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 02:00:00,27519.0,2348.0,3516.0,3410.0,599.0,-359.0,-165.0,-1.609396,6.846316,12.722727,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 03:00:00,27144.0,2298.0,3373.0,3275.0,578.0,-317.0,-141.0,-1.609396,6.846316,12.722727,87301.0,55923.0,NaN,NaN,NaN
2020-01-01 04:00:00,27164.0,2284.0,3091.0,3038.0,502.0,-189.0,-80.0,-1.609396,6.846316,12.722727,87301.0,55923.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-17 19:00:00,40221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 20:00:00,41317.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-17 21:00:00,41978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df=df.fillna(method='backfill')
df=df.dropna()
df

,price,consumption,production,da_production,exchange_NO3_NO4,exchange_NO4_SE1,exchange_NO4_SE2,air_temperature,wind_speed,precipitation_amount,reservoir_capacity,reservoir_levels,oil_price,gas_price,osebx_price
datetime,,,,,,,,,,,,,,,
2020-01-01 00:00:00,28387.0,2192.0,2111.0,2100.0,298.0,308.0,42.0,-1.609396,6.846316,12.722727,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 01:00:00,28062.0,2357.0,3544.0,3452.0,646.0,-333.0,-163.0,-1.609396,6.846316,12.722727,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 02:00:00,27519.0,2348.0,3516.0,3410.0,599.0,-359.0,-165.0,-1.609396,6.846316,12.722727,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 03:00:00,27144.0,2298.0,3373.0,3275.0,578.0,-317.0,-141.0,-1.609396,6.846316,12.722727,87301.0,55923.0,582.464933,18.656462,941.270020
2020-01-01 04:00:00,27164.0,2284.0,3091.0,3038.0,502.0,-189.0,-80.0,-1.609396,6.846316,12.722727,87301.0,55923.0,582.464933,18.656462,941.270020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 20:00:00,49655.0,2780.0,4455.0,4415.0,1190.0,-326.0,-120.0,-7.851653,3.291089,0.363125,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 21:00:00,48028.0,2817.0,4495.0,4379.0,1168.0,-299.0,-117.0,-7.851653,3.291089,0.363125,87301.0,53981.0,815.527912,20.742414,1168.589966
2023-03-29 22:00:00,37975.0,2881.0,4597.0,4388.0,1202.0,-344.0,-120.0,-7.851653,3.291089,0.363125,87301.0,53981.0,815.527912,20.742414,1168.589966


In [26]:
df.to_csv('NO4_preprocessed')